# State-Machine States

### State Name : Check for Transfomation Process (Geçiş Süreci Kontrolü)


Steps for Processing ContractID:

1. Initialize the ContractID. As the first step in the Business Mapping process, 
   invoke the `md/search` service to retrieve all relevant information. 
   Store the necessary data in a variable named `CONTEXT`.

2. Execute the post-execution script to perform all required controls. 
   If any errors are encountered during this process, return the relevant error message.

3. Update the `CONTEXT` variable according to the requirements of the subsequent steps.



##### Business Mapping

In [121]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}

def business_mapping(machine, state, request_data, context, prev_state_response, current_state_response):
    # do workflow here
    return {"success": True, "goto": "END"}

##### Post-Execution

In [123]:
def post_execution(machine, state, request_data, context, prev_state_response, current_state_response):
    
    condition = {
        'TermType': 'CT',
        'ContractCancellationDate': 99991231235959
    }
    
    if context['assetDefinitionCode'] not in ['ConsumerContract', 'LicensedProducerContract', 'UnlicensedProducerContract']:
        return {"success": False, "ErrorCode": "Üretici ya da tüketici değilseniz Terim değişikliği yapılamaz", "goto": "END"}
    
    # ConsumerContract için TariffCode kontrolü
    if context['assetDefinitionCode'] == 'ConsumerContract':
        tariff_code = next((attr['value'] for attr in context['attributes'] 
                            if attr['attribute']['attributeCode'] == 'TariffCode'), None)

        # Eğer tariff_code None ise, values'dan al
        if tariff_code is None:
            tariff_code = next((attr['values'][0]['value'] for attr in context['attributes'] 
                                if attr['attribute']['attributeCode'] == 'TariffCode' and attr['values']), None)
            if any('AG' in code for code in tariff_code):
                return {"success": False, "ErrorCode": "AG iseniz terim değişikliği yapılamaz", "goto": "END"}
    #Şimdi kalan kontrollerde sıra
    if all(next((attr['value'] if attr['value'] is not None else (attr['values'][0]['value'] if attr['values'] else None) 
                    for attr in context['attributes'] 
                    if attr['attribute']['attributeCode'] == key), None) == value for key, value in condition.items()):
        
        term_type_values = [attr for attr in context['attributes'] 
                            if attr['attribute']['attributeCode'] == 'TermType']
        
        if term_type_values:
            count = len(term_type_values[0].get('values', []))
            if count > 3:
                return {"success": False, "ErrorCode": "3'ten fazla terim değişikliği yapılamaz", 'goto': 'END'}        
            else:
                return {"success": True, "goto": "State 3"}
    else :
        return {"success": False, "ErrorCode": "Terim değişikliği yapmaya uygun değilsiniz", "goto": "END"}


##### Context Store Mapping

In [118]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def context_store_mapping(machine, state, source, context, prevStateResponse):
    store = {}
    
    attribute_codes = ['ServicePointId', 'BusinessPartnerId', 'AccountId']
    
    for code in attribute_codes:
        for attr in context['attributes']:
            if attr['attribute']['attributeCode'] == code:
                value = attr['value']
                if value is None and attr['values']:
                    value = attr['values'][0]['value']  # values içindeki value al
                store[code] = value
                break
        else:
            store[code] = None  # Eğer bulunamazsa None döner
    return store


In [125]:
import json
with open(r'C:\Users\KaanGermiyan\StateMachine\md-search-first-output.json') as file:
    context = json.load(file)

In [126]:
business_mapping('machine', 'state', 'request_data', context, 'prev_state_response', 'current_state_response')

post_execution('machine', 'state', 'request_data', context, 'prev_state_response', 'current_state_response')



{'success': False,
 'ErrorCode': 'Terim değişikliği yapmaya uygun değilsiniz',
 'goto': 'END'}

In [127]:
context_store_mapping('machine', 'state', 'source', context, 'prevStateResponse')

{'ServicePointId': 21407, 'BusinessPartnerId': None, 'AccountId': None}

In [1]:
#P.S:
# Here is a problem .. There is no AttributeCode as ! BusinessPartnerId and AccountId ! in the Json.. These two must be checked !